In [2]:
using LinearAlgebra, Random, DataFrames, CSV, Plots
using StatsBase, Statistics
using JuMP, Gurobi

In [7]:
Random.seed!(42)
# Paths (relative to notebook structure)
restaurant_path    = "../clean_data/restaurant_data_expanded.csv"
scrap_path         = "../clean_data/food_scrap_locations.csv"
neighborhood_path  = "../clean_data/neighborhood_supply.csv"

# Read CSVs into DataFrames
restaurant_data       = CSV.read(restaurant_path, DataFrame)
food_scrap_locations  = CSV.read(scrap_path, DataFrame)
neighborhood_supply   = CSV.read(neighborhood_path, DataFrame)
restaurant_data = restaurant_data[shuffle(1:nrow(restaurant_data))[1:500], :]

# Preview the first few rows
println(first(restaurant_data, 5))
println(first(food_scrap_locations, 5))
println(first(neighborhood_supply, 5))

5×3 DataFrame
 Row │ latitude  longitude  waste         
     │ Float64   Float64    Float64       
─────┼────────────────────────────────────
   1 │  40.7424   -73.9943      1.11117e5
   2 │  40.7323   -73.8722  71532.9
   3 │  40.7794   -73.9492  98561.5
   4 │  40.6102   -73.9208  19103.5
   5 │  40.7315   -73.861   94690.4
5×27 DataFrame
 Row │ Borough    NTAName                  SiteName                           SiteAddr                           Hosted_By                       Open_Month  Day_Hours                          Notes                      Website                            BoroCD  CouncilDis  ct2010   BBL       BIN      Latitude  Longitude  PolicePrec  Object.ID  Location.Point                App.Android  App.iOS  X.Assembly.District  X.Congress.District  DSNY.District  X.DSNY.Section  DSNY.Zone  Senate.District 
     │ String15   String                   String                             String                             String                          String31    

In [8]:
current_pantries       = CSV.read("../clean_data/Current Food Pantries.csv", DataFrame)

Row,Latitude,Longitude
,Float64,Float64
1,40.7541,-73.9933
2,40.6982,-73.9752
3,40.6964,-73.9911
4,40.7278,-74.0009
5,40.7513,-73.9953
6,40.8019,-73.9665


In [9]:
# ================================
# 3. CLEAN RESTAURANT
# ================================
# Columns: latitude | longitude | waste
rename!(restaurant_data, names(restaurant_data)[3] => :supply)

# Ensure Float64
restaurant_data.supply    = Float64.(restaurant_data.supply)
restaurant_data.latitude  = Float64.(restaurant_data.latitude)
restaurant_data.longitude = Float64.(restaurant_data.longitude)

# *** DROP RESTAURANTS WITH NEGATIVE SUPPLY ***
filter!(row -> row.supply >= 0, restaurant_data)



# ================================
# 3. CLEAN FOOD SCRAP CENTER DATA
# ================================
# rename Latitude and Longitude to latitude and longitude for consistency
rename!(food_scrap_locations, names(food_scrap_locations)[15] => :latitude)
rename!(food_scrap_locations, names(food_scrap_locations)[16] => :longitude)

# Keep only coordinates we need
food_scrap_locations.latitude = Float64.(food_scrap_locations.latitude)
food_scrap_locations.longitude = Float64.(food_scrap_locations.longitude)

# Keep only coordinate columns in food_scrap_locations
select!(food_scrap_locations, [:latitude, :longitude])

# ================================
# 4. CLEAN NEIGHBORHOOD SUPPLY DATA
# ================================
# Rename demand column for clarity
rename!(neighborhood_supply, names(neighborhood_supply)[4] => :supply_gap)

# Now neighborhood_supply.supply_gap might be String OR Float64.
# Only do replace/parse if it's strings.
if eltype(neighborhood_supply.supply_gap) <: AbstractString
    neighborhood_supply.supply_gap =
        parse.(Float64, replace.(neighborhood_supply.supply_gap, "," => ""))
end

# Demand = positive deficit, surplus -> 0
neighborhood_supply.demand = max.(0.0, -neighborhood_supply.supply_gap)

neighborhood_supply.latitude  = Float64.(neighborhood_supply.latitude)
neighborhood_supply.longitude = Float64.(neighborhood_supply.longitude)

# keep only necessary columns from neighborhood supply
select!(neighborhood_supply, [:latitude, :longitude, :demand, :Year])

# ================================
# 5. SHOW CLEANED HEADS
# ================================
println("=== Restaurants (cleaned) ===")
println(first(restaurant_data, 5))

println("\n=== Food Scrap Locations (cleaned) ===")
println(first(food_scrap_locations, 5))

println("\n=== Neighborhood Supply (cleaned) ===")
println(first(neighborhood_supply, 5))

=== Restaurants (cleaned) ===
5×3 DataFrame
 Row │ latitude  longitude  supply        
     │ Float64   Float64    Float64       
─────┼────────────────────────────────────
   1 │  40.7424   -73.9943      1.11117e5
   2 │  40.7323   -73.8722  71532.9
   3 │  40.7794   -73.9492  98561.5
   4 │  40.6102   -73.9208  19103.5
   5 │  40.7315   -73.861   94690.4

=== Food Scrap Locations (cleaned) ===
5×2 DataFrame
 Row │ latitude  longitude 
     │ Float64   Float64   
─────┼─────────────────────
   1 │  40.6355   -74.0228
   2 │  40.7526   -73.969
   3 │  40.7635   -74.0002
   4 │  40.762    -73.9693
   5 │  40.7174   -74.0108

=== Neighborhood Supply (cleaned) ===
5×4 DataFrame
 Row │ latitude  longitude  demand     Year  
     │ Float64   Float64    Float64    Int64 
─────┼───────────────────────────────────────
   1 │  40.8267   -73.9217  1.02143e5   2025
   2 │  40.8321   -73.8887  3.33493e5   2025
   3 │  40.8016   -73.9374  0.0         2025
   4 │  40.8469   -73.8918  1.13653e5   202

In [10]:
# ================
# 6. EXTRACT VECTORS
# ================
supply = restaurant_data.supply              # s_i
demand = neighborhood_supply.demand          # d_k

R = nrow(restaurant_data)
D = nrow(current_pantries)
N = nrow(neighborhood_supply[neighborhood_supply.Year .== 2023, :])
Q = sum(restaurant_data.supply)

M = sum(demand)
F = 10000       # Center opening cost

S = 3 # 3 demand years




# ================
# 7. MANHATTAN DISTANCE FUNCTION
# ================
manhattan(lat1, lon1, lat2, lon2) = abs(lat1 - lat2) + abs(lon1 - lon2)

# ================
# 8. COST MATRICES
# ================
# cij: Restaurants (i) → Donation centers (j)
cij = [manhattan(restaurant_data.latitude[i], restaurant_data.longitude[i],
                 current_pantries.Latitude[j], current_pantries.Longitude[j])
       for i in 1:R, j in 1:D]

# cjk: Donation centers (j) → Neighborhoods (k)
cjk = [manhattan(current_pantries.Latitude[j], current_pantries.Longitude[j],
                 neighborhood_supply.latitude[k], neighborhood_supply.longitude[k])
       for j in 1:D, k in 1:N]

println("Size of cij (R x D): ", size(cij))
println("Size of cjk (D x N): ", size(cjk))


Size of cij (R x D): (500, 6)
Size of cjk (D x N): (6, 197)


In [11]:


demand_2023 = neighborhood_supply[neighborhood_supply.Year .== 2023, :].demand
demand_2024 = neighborhood_supply[neighborhood_supply.Year .== 2024, :].demand
demand_2025 = neighborhood_supply[neighborhood_supply.Year .== 2025, :].demand

demand_df = DataFrame(
    y2023 = demand_2023,
    y2024 = demand_2024,
    y2025 = demand_2025
)

Row,y2023,y2024,y2025
,Float64,Float64,Float64
1,0.0,0.0,1.02143e5
2,0.0,0.0,3.33493e5
3,0.0,0.0,0.0
4,0.0,0.0,1.13653e5
5,0.0,0.0,0.0
6,82217.3,13128.3,5.06273e5
7,2.25651e5,0.0,0.0
8,9.04474e5,0.0,1.13911e5
9,0.0,0.0,0.0


In [12]:
average_demand = mean(Matrix(demand_df), dims=2)

197×1 Matrix{Float64}:
  34047.54142098
 111164.21622397
      0.0
  37884.271552896666
      0.0
 200539.58538104015
  75216.89106666666
 339461.9416397367
      0.0
      1.2123206961212999e6
      0.0
 124455.17158937
 660935.3033488033
      ⋮
      0.0
 477478.5816756173
 243880.36077310666
      0.0
      0.0
 773510.1512916334
      0.0
      0.0
 103669.25587499166
 776679.7817636967
      1.14574152677609e6
 265731.63622191997

In [13]:
println("Total supply: ", sum(supply))
println("Total demand (avg): ", mean([sum(demand_df[:,s]) for s in 1:S]))
println("Supply/Demand ratio: ", sum(supply) / mean([sum(demand_df[:,s]) for s in 1:S]))

Total supply: 3.729225025921673e7
Total demand (avg): 5.081311007183481e7
Supply/Demand ratio: 0.7339100127210566


In [14]:
function build_combined_model(w_cost::Float64, w_eq::Float64, year)
    model = Model(Gurobi.Optimizer)
    set_silent(model)

    # Decision variables
    @variable(model, x[1:R, 1:D] >= 0)   # restaurant -> center
    @variable(model, y[1:D, 1:N] >= 0)   # center -> neighborhood
    @variable(model, u[1:N] >= 0)        # unmet demand (for cost part)
    @variable(model, r[1:N] >= 0)        # received (for equity part)
    @variable(model, t >= 0)             # worst unmet demand

    # === Shared constraints ===

    # 1) Restaurant supply
    @constraint(model, [i in 1:R],
        sum(x[i,j] for j in 1:D) == supply[i]
    )

    # 2) Flow conservation at centers
    @constraint(model, [j in 1:D],
        sum(y[j,k] for k in 1:N) == sum(x[i,j] for i in 1:R)
    )

    # 4) r_k = received = ∑_j y_jk
    @constraint(model, [k in 1:N],
        r[k] == sum(y[j,k] for j in 1:D)
    )

    # (optional, redundant but harmless) r_k ≤ d_k
    @constraint(model, [k in 1:N],
        r[k] <= demand_df[k,year]
    )

    # 5) Worst unmet demand: t ≥ d_k - r_k
    @constraint(model, [k in 1:N],
        t >= demand_df[k, year] - r[k]
    )

    # === Objective pieces ===

    @expression(model, cost_expr,
        sum(cij[i,j] * x[i,j] for i in 1:R, j in 1:D) +
        sum(cjk[j,k] * y[j,k] for j in 1:D, k in 1:N)
    )

    @expression(model, equity_expr, t)

    # Weighted objective: w_cost * cost + w_eq * equity
    @objective(model, Min, w_cost * cost_expr + w_eq * equity_expr)

    return model, x, y, u, r, t, cost_expr, equity_expr
end

build_combined_model (generic function with 1 method)

In [15]:
function compute_stats(model, x, y, u, r, t)

    # Solve the model
    optimize!(model)

    # === Extract solution ===
    x_val = value.(x)
    y_val = value.(y)
    u_val = value.(u)
    r_val = value.(r)
    t_val = value(t)

    # === Total quantities ===
    total_flow_rest_to_center = sum(x_val)
    total_flow_center_to_neighborhood = sum(y_val)
    total_unmet = sum(u_val)

    # === Worst-case unmet ===
    worst_unmet = t_val

    # === Distance / Cost metrics ===
    total_distance_ij = sum(cij[i,j] * x_val[i,j] for i in 1:R, j in 1:D)
    total_distance_jk = sum(cjk[j,k] * y_val[j,k] for j in 1:D, k in 1:N)
    total_distance = total_distance_ij + total_distance_jk

    # === Per-neighborhood stats ===
    received = r_val
    unmet = u_val
    demand_vs_received = [ (average_demand[k], received[k], unmet[k]) for k in 1:N ]

    # === Pack results ===
    return (
        total_flow_rest_to_center = total_flow_rest_to_center,
        total_flow_center_to_neighborhood = total_flow_center_to_neighborhood,
        total_unmet = total_unmet,
        worst_unmet = worst_unmet,
        total_distance_ij = total_distance_ij,
        total_distance_jk = total_distance_jk,
        total_distance = total_distance,
        received = received,
        unmet = unmet,
        demand_vs_received = demand_vs_received,
        objective_value = objective_value(model)
    )
end


compute_stats (generic function with 1 method)

In [16]:
results = DataFrame(
    Year = Int[],
    TotalDistance = Float64[],
    WorstUnmetDemand = Float64[]
)

for yr in 1:3
    model, x, y, u, r, t, cost_expr, equity_expr = build_combined_model(1.0, 2.0, yr)
    optimize!(model)
    
    stats = compute_stats(model, x, y, u, r, t)
    
    push!(results, (
        yr,
        stats.total_distance,
        stats.worst_unmet
    ))
end

results


Set parameter Username
Set parameter LicenseID to value 2697112
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Set parameter LicenseID to value 2697112
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Set parameter LicenseID to value 2697112
Academic license - for non-commercial use only - expires 2026-08-20


Row,Year,TotalDistance,WorstUnmetDemand
,Int64,Float64,Float64
1,1,6.68561e6,1.91268e5
2,2,6.61305e6,7.29219e5
3,3,7.18893e6,4.95528e5


In [17]:
results.Year = ["2023","2024","2025"]

3-element Vector{String}:
 "2023"
 "2024"
 "2025"

In [18]:
results

Row,Year,TotalDistance,WorstUnmetDemand
,String,Float64,Float64
1,2023,6.68561e6,1.91268e5
2,2024,6.61305e6,7.29219e5
3,2025,7.18893e6,4.95528e5


In [19]:
CSV.write("viz_data/baseline_deterministic_solution.csv", results)

"viz_data/baseline_deterministic_solution.csv"

In [50]:
run(`git push`)

To https://github.com/lauren-montigue/Optimizing-Food-Waste-Distribution-NY.git
   b9d6035..1f58ad9  main -> main


Process(`git push`, ProcessExited(0))